In [3]:
# this file will be used to train and
# report accuracy results of the model

import os
import copy
import time
import pickle

import numpy as np #numbers
from tqdm import tqdm #progress bar
import torch #pytorch
from tensorboardX import SummaryWriter #display metrics

In [5]:

from torch import nn

class Network(nn.Module):

    def __init__(self):
        super().__init__()

        # Inputs to hidden layer linear transformation
        self.hidden = nn.Linear(3, 5)
        # Output layer, 10 units - one for each digit
        self.output = nn.Linear(5, 2)
        # Define sigmoid activation and softmax output
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.output(x)
        x = self.softmax(x)

        return x

model = Network()
model


Network(
  (hidden): Linear(in_features=3, out_features=5, bias=True)
  (output): Linear(in_features=5, out_features=2, bias=True)
  (sigmoid): Sigmoid()
  (softmax): Softmax(dim=1)
)